In [1]:
!pip install git+https://github.com/openai/CLIP.git
!pip install kaolin==0.17.0 -f https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.5.1_cu121.html
!pip install open-clip-torch


!pip install open3d


!git clone https://github.com/AlesCarl/AML_group.git
%cd AML_group


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ik8d2xze
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ik8d2xze
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.5 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=fa4616c58f795e9fdc557b3b1d59a710130c2c568b8c3fa0547d0c0cc030a511
  Stored in directory: /tmp/pip-ephem-wheel-cache-cozi05dr/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
Looking in links: https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.5.1_cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━

In [11]:
import clip
import copy
import json
import kaolin as kal
import kaolin.ops.mesh
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torchvision
import open_clip
import open3d as o3d


from itertools import permutations, product
from Normalization import MeshNormalizer


from mesh import Mesh
from pathlib import Path
from render import Renderer
from tqdm import tqdm
from torch.autograd import grad
from torchvision import transforms
from utils import device, color_mesh

class NeuralHighlighter(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(NeuralHighlighter, self).__init__()

        layers = []
        for i in range(num_layers):
            layers.append(nn.Linear(input_dim if i == 0 else hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.LayerNorm([hidden_dim]))

        layers.append(nn.Linear(hidden_dim, output_dim))
        layers.append(nn.Softmax(dim=1))

        self.mlp = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.mlp:
            x = layer(x)
        return x


def get_clip_model(clip_model):
    device = 'cuda'
    model, preprocess = clip.load(clip_model, device=device) # jit = True for better perfomance
    return model



# ================== HELPER FUNCTIONS =============================
def save_final_results(log_dir, name, mesh, mlp, vertices, colors, render, background):
    mlp.eval()
    with torch.no_grad():
        probs = mlp(vertices)
        max_idx = torch.argmax(probs, 1, keepdim=True)
        # for renders
        one_hot = torch.zeros(probs.shape).to(device)
        one_hot = one_hot.scatter_(1, max_idx, 1)
        sampled_mesh = mesh

        highlight = torch.tensor([204, 255, 0]).to(device)
        gray = torch.tensor([180, 180, 180]).to(device)
        colors = torch.stack((highlight/255, gray/255)).to(device)
        color_mesh(one_hot, sampled_mesh, colors)
        rendered_images, _, _ = render.render_views(sampled_mesh, num_views=5,
                                                                        show=False,
                                                                        center_azim=0,
                                                                        center_elev=0,
                                                                        std=1,
                                                                        return_views=True,
                                                                        lighting=True,
                                                                        background=background)
        # for mesh
        final_color = torch.zeros(vertices.shape[0], 3).to(device)
        final_color = torch.where(max_idx==0, highlight, gray)
        mesh.export(os.path.join(log_dir, f"{name}.ply"), extension="ply", color=final_color)
        save_renders(log_dir, 0, rendered_images, name='final_render.jpg')


def clip_loss(n_augs, rendered_images, encoded_text, clip_transform, augment_transform, clip_model):

    if n_augs == 0:
        clip_image = clip_transform(rendered_images)
        encoded_renders = clip_model.encode_image(clip_image)
        encoded_renders = encoded_renders / encoded_renders.norm(dim=1, keepdim=True)
        if encoded_text.shape[0] > 1:
            loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                torch.mean(encoded_text, dim=0), dim=0)
        else:
            loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                encoded_text)
    elif n_augs > 0:
        loss = 0.0
        for _ in range(n_augs):
            augmented_image = augment_transform(rendered_images)
            encoded_renders = clip_model.encode_image(augmented_image)
            if encoded_text.shape[0] > 1:
                loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                    torch.mean(encoded_text, dim=0), dim=0)
            else:
                loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                    encoded_text)
    return loss
    # "1-loss" removed -> now best value in output is -1

def save_renders(dir, i, rendered_images, name=None):
    if name is not None:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, name))
    else:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, 'renders/iter_{}.jpg'.format(i)))


In [ ]:
# Constrain most sources of randomness
# (some torch backwards functions within CLIP are non-determinstic)
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


render_res = 224
n_iter = 1000
res = 224
obj_path = 'data/horse.obj' # 'APPROXIMATE.obj' if we want to test the point cloud to mesh conversion
output_dir = './output/'
clip_model = 'ViT-L/14'

input_dim = 3
hidden_dim = 256
output_dim = 2

# Hyper-parameters
learning_rate = 0.0001
n_layers = 3 # depth 4
n_views = 3 # 5
n_augs = 1 # 4

Path(os.path.join(output_dir, 'renders')).mkdir(parents=True, exist_ok=True)

objbase, extension = os.path.splitext(os.path.basename(obj_path))

render = Renderer(dim=(render_res, render_res))
mesh = Mesh(obj_path)
MeshNormalizer(mesh)()


### --

# Initialize variables
background = torch.tensor((1., 1., 1.)).to(device)

log_dir = output_dir


# CLIP and augmentation transform
clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
clip_transform = transforms.Compose([
    transforms.Resize((res, res)),
    clip_normalizer
])
augment_transform = transforms.Compose([
    transforms.RandomResizedCrop(res, scale=(1, 1)),
    transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5),
    clip_normalizer
])



# MLP Settings
mlp = NeuralHighlighter(input_dim, hidden_dim, output_dim, n_layers).to(device)
optim = torch.optim.Adam(mlp.parameters(), learning_rate)

# list of possible colors
rgb_to_color = {(204/255, 1., 0.): "highlighter", (180/255, 180/255, 180/255): "gray"}
color_to_rgb = {"highlighter": [204/255, 1., 0.], "gray": [180/255, 180/255, 180/255]}
full_colors = [[204/255, 1., 0.], [180/255, 180/255, 180/255]]
colors = torch.tensor(full_colors).to(device)


# --- Prompt ---
# encode prompt with CLIP
model = get_clip_model(clip_model)

known_object = 'horse'
classes = 'Shoes'


prompt = "A 3D render of a gray {} with highlighted {}".format(known_object, classes)
with torch.no_grad():
    prompt_token = clip.tokenize([prompt]).to(device)
    encoded_text = model.encode_text(prompt_token)
    encoded_text = encoded_text / encoded_text.norm(dim=1, keepdim=True)


vertices = copy.deepcopy(mesh.vertices)

losses = []

# Optimization loop
for i in tqdm(range(n_iter)):
    optim.zero_grad()

    # predict highlight probabilities
    pred_class = mlp(vertices)

    # color and render mesh
    sampled_mesh = mesh
    color_mesh(pred_class, sampled_mesh, colors)
    rendered_images, elev, azim = render.render_views(sampled_mesh, num_views=n_views,
                                                            show=False,
                                                            center_azim=0,
                                                            center_elev=0,
                                                            std=1,
                                                            return_views=True,
                                                            lighting=True,
                                                            background=background)

    # Calculate CLIP Loss
    loss = clip_loss(n_augs, rendered_images, encoded_text, clip_transform, augment_transform, model)
    loss.backward(retain_graph=True)

    optim.step()

    # update variables + record loss
    with torch.no_grad():
        losses.append(loss.item())

    # report results
    if i % 100 == 0:
        print("Last 100 CLIP score: {}".format(np.mean(losses[-100:])))
        save_renders(log_dir, i, rendered_images)
        with open(os.path.join(log_dir, "training_info.txt"), "a") as f:
            f.write(f"For iteration {i}... Prompt: {prompt}, Last 100 avg CLIP score: {np.mean(losses[-100:])}, CLIP score {losses[-1]}\n")


# save results
save_final_results(log_dir, 'Primo test', mesh, mlp, vertices, colors, render, background)

# Save prompts
with open(os.path.join(dir, prompt), "w") as f:
    f.write('')

  0%|          | 1/1000 [00:01<28:29,  1.71s/it]

Last 100 CLIP score: -0.2398681640625


 10%|█         | 101/1000 [00:22<03:45,  3.99it/s]

Last 100 CLIP score: -0.30198974609375


 20%|██        | 201/1000 [00:43<02:49,  4.71it/s]

...

Last 100 CLIP score: -0.3236083984375


 90%|█████████ | 901/1000 [03:15<00:21,  4.58it/s]

Last 100 CLIP score: -0.319642333984375


100%|██████████| 1000/1000 [03:37<00:00,  4.60it/s]


TypeError: expected str, bytes or os.PathLike object, not builtin_function_or_method

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
# Percorso del file OBJ
obj_path = 'data/horse.obj'

# Funzione per caricare la mesh come TriangleMesh
def load_obj_as_triangle_mesh(obj_path):
    # Carica il file .obj
    mesh = o3d.io.read_triangle_mesh(obj_path)
    if mesh.is_empty():
        raise ValueError(f"La mesh nel file {obj_path} non è stata caricata correttamente.")
    return mesh

# Carica la mesh
mesh = load_obj_as_triangle_mesh(obj_path)

# Converte la mesh in una point cloud campionando punti uniformemente
pcd = mesh.sample_points_uniformly(2048)

# Imposta il colore nero per tutti i punti
pcd.colors = o3d.utility.Vector3dVector(np.zeros((len(pcd.points), 3)))  # Colore nero: [0, 0, 0]


# Esporta la point cloud in formato PLY
# o3d.io.write_point_cloud("candle_PC.ply", pcd)

In [28]:
# Point cloud to mesh

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)

o3d.io.write_triangle_mesh("APPROXIMATE.obj", mesh)

[Open3D DEBUG] Input Points / Samples: 2048 / 2038
[Open3D DEBUG] #   Got kernel density: 0.01277017593383789 (s), 3829.7890625 (MB) / 3829.7890625 (MB) / 3829 (MB)
[Open3D DEBUG] #     Got normal field: 0.008168935775756836 (s), 3831.078125 (MB) / 3831.078125 (MB) / 3831 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 5.874402e-04 / 1.203078e+00
[Open3D DEBUG] #       Finalized tree: 0.024731159210205078 (s), 3831.3359375 (MB) / 3831.3359375 (MB) / 3831 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.1672039031982422 (s), 3831.3359375 (MB) / 3831.3359375 (MB) / 3831 (MB)
[Open3D DEBUG] #Set point constraints: 0.00595402717590332 (s), 3831.3359375 (MB) / 3831.3359375 (MB) / 3831 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 153140 / 85672 / 89345
[Open3D DEBUG] Memory Usage: 3831.336 MB
[Open3D DEBUG] # Linear system solved: 0.13187098503112793 (s), 3831.3359375 (MB) / 3831.3359375 (MB) / 3831 (MB)
[Open3D DEBUG] Got average: 0.00551605224609375 (s), 3831.3359375 (MB) /

True

3 part

In [ ]:
!pip install transforms3d


from transforms3d.quaternions import quat2mat
from transforms3d.euler import euler2mat



def random_rotation_matrix():
    rand = np.random.rand(3)
    r1 = np.sqrt(1.0 - rand[0])
    r2 = np.sqrt(rand[0])
    pi2 = np.pi * 2.0
    t1 = pi2 * rand[1]
    t2 = pi2 * rand[2]
    q = np.array([np.cos(t2)*r2, np.sin(t1)*r1, np.cos(t1)*r1, np.sin(t2)*r2])
    return quat2mat(q)




def rotate_point_cloud_SO3(batch_data):
    rotated_data = np.zeros(batch_data.shape, dtype=np.float32)
    for k in range(batch_data.shape[0]):
        rotation_matrix = random_rotation_matrix()
        shape_pc = batch_data[k, ...]
        rotated_data[k, ...] = (
            np.matmul(rotation_matrix, shape_pc.reshape((-1, 3)).T)).T
    return rotated_data


def rotate_point_cloud_y(batch_data):

    rotated_data = np.zeros(batch_data.shape, dtype=np.float32)
    for k in range(batch_data.shape[0]):
        rotation_angle = np.random.uniform() * 2 * np.pi
        rotation_matrix = euler2mat(0, rotation_angle, 0)
        shape_pc = batch_data[k, ...]
        rotated_data[k, ...] = (
            np.matmul(rotation_matrix, shape_pc.reshape((-1, 3)).T)).T
    return rotated_data

In [66]:
import os
from os.path import join as opj
import numpy as np
from torch.utils.data import Dataset
import h5py
import json
import pickle as pkl


def pc_normalize(pc):
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc, centroid, m


class AffordNetDataset(Dataset):
    def __init__(self, data_dir, split):
        super().__init__()
        self.data_dir = data_dir
        self.split = split

        self.load_data()

        self.affordance = self.all_data[0]["affordance"]

        return

    def load_data(self):
     self.all_data = []
     with open(opj(self.data_dir, 'full_shape_%s_data.pkl' % self.split), 'rb') as f:
        temp_data = pkl.load(f)
     for index, info in enumerate(temp_data):
        if info["semantic class"] == "Bowl":  # Filtra solo gli oggetti "BOWL"
            temp_info = {}
            temp_info["shape_id"] = info["shape_id"]
            temp_info["semantic class"] = info["semantic class"]
            temp_info["affordance"] = info["affordance"]
            temp_info["data_info"] = info["full_shape"] # vertici
            self.all_data.append(temp_info)




    def __getitem__(self, index):

        data_dict = self.all_data[index]
        modelid = data_dict["shape_id"]
        modelcat = data_dict["semantic class"]

        data_info = data_dict["data_info"]
        model_data = data_info["coordinate"].astype(np.float32)
        labels = data_info["label"]
        for aff in self.affordance:
            temp = labels[aff].astype(np.float32).reshape(-1, 1)
            model_data = np.concatenate((model_data, temp), axis=1)

        datas = model_data[:, :3]
        targets = model_data[:, 3:]

        datas, _, _ = pc_normalize(datas)

        return datas, datas, targets, modelid, modelcat

    def __len__(self):
        return len(self.all_data)

In [4]:
from torch.utils.data import DataLoader
from torch.utils.data import Subset

def build_dataset(data_dir, test=False):
    test_set = AffordNetDataset( data_dir, 'train')
    val_set = AffordNetDataset( data_dir, 'val')

     # 5 campioni per il VAL set
    val_indices = list(range(min(5, len(val_set))))
    val_set = Subset(val_set, val_indices)

    # 10 campioni per il test set
    test_indices = list(range(min(10, len(test_set))))
    test_set = Subset(test_set, test_indices)

    dataset_dict = dict( val_set=val_set, test_set=test_set)
    return dataset_dict


def build_loader(dataset_dict):
    val_set = dataset_dict["val_set"]
    test_set = dataset_dict["test_set"]

    batch_size_factor = 1


    test_loader = DataLoader(test_set, batch_size=8, ## occhio qui
                              shuffle=True, drop_last=True, num_workers=8)

    val_loader = DataLoader(val_set, batch_size=1,
                            shuffle=False, num_workers=8, drop_last=False)
    loader_dict = dict(
        val_loader=val_loader,
        test_loader=test_loader ,
    )

    return loader_dict

In [ ]:
import torch
import numpy as np
from os.path import join as opj
from tqdm import tqdm
from sklearn.metrics import average_precision_score, f1_score, roc_auc_score


def evaluation(model, test_loader, affordance):
    results = torch.zeros(
        (len(test_loader), 2048, len(affordance)))
    targets = torch.zeros(
        (len(test_loader), 2048, len(affordance)))
    coordinate = np.zeros((0, 2048, 3))
    modelids = []
    modelcats = []
    with torch.no_grad():
        model.eval()
        total_L2distance = 0
        count = 0.0
        for i,  temp_data in tqdm(enumerate(test_loader), total=len(test_loader), smoothing=0.9):

            # modelcat identifies the category of the 3d model we are passing to our model
            data, data1, label, modelid, modelcat = temp_data

            data, label = data.float().cuda(), label.float().cuda()
            data = data.permute(0, 2, 1)
            batch_size = data.size()[0]
            num_point = data.size()[2]
            count += batch_size * num_point
            afford_pred = torch.sigmoid(model(data))
            afford_pred = afford_pred.permute(0, 2, 1).contiguous()
            L2distance = torch.sum(
                torch.pow(label-afford_pred, 2), dim=(0, 1))
            total_L2distance += L2distance
            score = afford_pred.squeeze()
            target_score = label.squeeze()
            results[i, :, :] = score
            targets[i, :, :] = target_score
            modelids.append(modelid[0])
            modelcats.append(modelcat[0])

    results = results.detach().cpu().numpy()
    targets = targets.detach().cpu().numpy()
    IOU = np.zeros((targets.shape[0], targets.shape[2]))
    targets = targets >= 0.5
    targets = targets.astype(int)

    IOU_thres = np.linspace(0, 1, 20)
    for i in range(IOU.shape[0]):
        t = targets[i, :, :]
        p = results[i, :, :]
        for j in range(t.shape[1]):
            t_true = t[:, j]
            p_score = p[:, j]
            if np.sum(t_true) == 0:
                IOU[i, j] = np.nan
            else:
                p_mask = (p_score > 0.5).astype(int)
                temp_iou = []
                for thre in IOU_thres:
                    p_mask = (p_score >= thre).astype(int)
                    intersect = np.sum(p_mask & t_true)
                    union = np.sum(p_mask | t_true)
                    temp_iou.append(1.*intersect/union)
                temp_iou = np.array(temp_iou)
                aiou = np.mean(temp_iou)
                IOU[i, j] = aiou

    IOU = np.nanmean(IOU, axis=0)

    outstr = 'Test :: test maIOU: %.6f' % (
        np.mean(IOU))

    return np.mean(IOU)

In [67]:
dataset_dir = '/content/drive/My Drive/full-shape'

dataset = build_dataset(dataset_dir)

loader = build_loader(dataset)





In [ ]:
# Assumendo che `loader["val_loader"]` sia il DataLoader principale
for batch in loader["val_loader"]:
    # Estraggo il primo elemento del batch
    datas, prova, targets, modelid, modelcat = batch

    # Stampare la "affordance" dal primo elemento
    print("Affordance:", prova[0])
    break  # Uscire dopo il primo batch


Affordance: tensor([[ 0.2516, -0.0257,  0.2042],
        [-0.6823,  0.6966, -0.2069],
        [-0.6833, -0.5620, -0.2057],
        ...,
        [-0.3412,  0.1812,  0.2042],
        [-0.0785, -0.5747, -0.1798],
        [ 0.3324, -0.5747,  0.1575]])


In [ ]:


import open3d as o3d
import numpy as np


In [89]:
# conversione point cloud to mesh:

def pointToMesh(data ):

    single_data = data[0].cpu().numpy()     # shape [N, 3]
    single_target = targets[0].cpu().numpy() # shape [N, num_affordances].     PER COSA LO USA

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(single_data)

    # new_points = np.random.normal(loc=np.array(pcd.points), scale=0.005, size=(len(pcd.points), 3))
    # dense_pcd = o3d.geometry.PointCloud()
    # dense_pcd.points = o3d.utility.Vector3dVector(np.vstack((np.array(pcd.points), new_points)))

    # (Opzionale) Se hai colori (o label) associati a ogni punto, potresti
    #  impostarli come pcd.colors, ma qui li useremo solo come reference.

    # Stima delle normali
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
        radius=0.08,  # Raggio per la stima delle normali, aumentalo se i punti sono più distanti
        max_nn=50     # Numero massimo di vicini per stimare la normale
    ))

    # Orienta le normali in modo consistente
    pcd.orient_normals_consistent_tangent_plane(k=30)

    # Calcola la distanza media tra i punti per ottimizzare il raggio della sfera
    distances = pcd.compute_nearest_neighbor_distance()
    mean_distance = np.mean(distances)
    print(f"Distanza media tra i punti: {mean_distance:.4f}")


    # Imposta i raggi per Ball Pivoting basandoti sulla densità della point-cloud
    radii = [mean_distance * 1.5, mean_distance * 2, mean_distance * 2.5]

    # Ricostruzione con Ball Pivoting
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        pcd, o3d.utility.DoubleVector(radii)
    )

    # (Opzionale) Liscia la mesh per rimuovere artefatti
    mesh = mesh.filter_smooth_laplacian(number_of_iterations=5)

    # Salva e visualizza la mesh
    o3d.io.write_triangle_mesh("bowl_mesh_ball_pivoting.obj", mesh)




    # Point cloud to mesh

# FUNZIONE

In [8]:



def optimize (vertex, targets , modelCat):

    pointToMesh(vertex) #


    seed = 42
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


    render_res = 224
    n_iter = 1000
    res = 224
    obj_path = 'data.obj'
    output_dir = './output/'
    clip_model = 'ViT-L/14'

    input_dim = 3
    hidden_dim = 256
    output_dim = 2

    # Hyper-parameters
    learning_rate = 0.0001
    n_layers = 4 # depth 4
    n_views = 3 # 5
    n_augs = 4 # 4

    Path(os.path.join(output_dir, 'renders')).mkdir(parents=True, exist_ok=True)

    objbase, extension = os.path.splitext(os.path.basename(obj_path))

    render = Renderer(dim=(render_res, render_res))
    mesh = Mesh(obj_path)
    MeshNormalizer(mesh)()



    # Initialize variables
    background = torch.tensor((1., 1., 1.)).to(device)

    log_dir = output_dir


    # CLIP and augmentation transform
    clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
    clip_transform = transforms.Compose([
        transforms.Resize((res, res)),
        clip_normalizer
    ])
    augment_transform = transforms.Compose([
        transforms.RandomResizedCrop(res, scale=(1, 1)),
        transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5),
        clip_normalizer
    ])



    # MLP Settings
    mlp = NeuralHighlighter(input_dim, hidden_dim, output_dim, n_layers).to(device)
    optim = torch.optim.Adam(mlp.parameters(), learning_rate)

    # list of possible colors
    rgb_to_color = {(204/255, 1., 0.): "highlighter", (180/255, 180/255, 180/255): "gray"}
    color_to_rgb = {"highlighter": [204/255, 1., 0.], "gray": [180/255, 180/255, 180/255]}
    full_colors = [[204/255, 1., 0.], [180/255, 180/255, 180/255]]
    colors = torch.tensor(full_colors).to(device)


    # --- Prompt ---
    # encode prompt with CLIP
    model = get_clip_model(clip_model)

    known_object = modelCat
    classes = "region for grasping"  # o una affordance specifica ----- DA SISTEMARE


    prompt = f"A 3D render of a gray {known_object} with highlighted {classes}"



    '''
    # Qui puoi definire i tuoi prompt CLIP
    known_object = modelcat[0]  # per esempio 'Bag'
    classes = "region for grasping"  # o una affordance specifica
    prompt = f"A 3D render of a gray {known_object} with highlighted {classes}"
    '''



    #prompt = "A 3D render of a gray {} with highlighted {}".format(known_object, classes)
    with torch.no_grad():
        prompt_token = clip.tokenize([prompt]).to(device)
        encoded_text = model.encode_text(prompt_token)
        encoded_text = encoded_text / encoded_text.norm(dim=1, keepdim=True)


    vertices = copy.deepcopy(mesh.vertices)

    losses = []

    # Optimization loop
    for i in tqdm(range(n_iter)):
        optim.zero_grad()

        # predict highlight probabilities
        pred_class = mlp(vertices)

        # color and render mesh
        sampled_mesh = mesh
        color_mesh(pred_class, sampled_mesh, colors)
        rendered_images, elev, azim = render.render_views(sampled_mesh, num_views=n_views,
                                                                show=False,
                                                                center_azim=0,
                                                                center_elev=0,
                                                                std=1,
                                                                return_views=True,
                                                                lighting=True,
                                                                background=background)

        # Calculate CLIP Loss
        loss = clip_loss(n_augs, rendered_images, encoded_text, clip_transform, augment_transform, model)
        loss.backward(retain_graph=True)

        optim.step()

        # update variables + record loss
        with torch.no_grad():
            losses.append(loss.item())

        # report results
        if i % 100 == 0:
            print("Last 100 CLIP score: {}".format(np.mean(losses[-100:])))
            save_renders(log_dir, i, rendered_images)
            with open(os.path.join(log_dir, "training_info.txt"), "a") as f:
                f.write(f"For iteration {i}... Prompt: {prompt}, Last 100 avg CLIP score: {np.mean(losses[-100:])}, CLIP score {losses[-1]}\n")


    # save results
    save_final_results(log_dir, 'TTest2', mesh, mlp, vertices, colors, render, background)


    # Save prompts
    # with open(os.path.join(dir, prompt), "w") as f:
      #  f.write('')






In [9]:
# puo essere utile in futuro:

known_object = "Bag"


affordances = {
    "grasp": "where a hand can grasp it",
    "push": "where a hand can push it",
    "wrap": "where fingers can wrap around it"
}

for affordance, description in affordances.items():
    prompt = f"A 3D render of a gray {known_object} with highlighted {description}"
    print(prompt)



''' NOTA BENE:


CLIP funziona bene con prompt che sembrano frasi naturali, quindi è meglio evitare descrizioni troppo tecniche o astratte come
"region for grasping". Scegli formulazioni che un umano userebbe per descrivere le affordance, ad esempio "where it can be grasped"
o "graspable areas". Questo può migliorare il risultato del tuo modello.

'''

NameError: name 'known_object' is not defined

In [90]:
for batch_idx, batch in enumerate(loader["val_loader"]):
    # if batch_idx == 0:
    #    continue
    # Estraggo il primo elemento del batch
    data, data1, targets, modelid, modelcat = batch   # con target = GT  ( area giusta da colorare )
    pointToMesh(data) #
    label = "grasp"

    #optimize(data, targets,modelcat, label)

    break



Distanza media tra i punti: 0.0574
[Open3D WARNING] GLFW Error: Failed to detect any supported platform
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [ ]:

# Numero di oggetti nel validation loader
num_val_objects = len(loader["val_loader"].dataset)
print(f"Number of objects in val_loader: {num_val_objects}")

# Numero di oggetti nel test loader
num_test_objects = len(loader["test_loader"].dataset)
print(f"Number of objects in test_loader: {num_test_objects}")


Number of objects in val_loader: 5
Number of objects in test_loader: 10


# TODO: domani:

- OTTENERE UN MESH DI QUALITA' MAGGIORE  !!